In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=1c1fb5d016824c6f93a3d12bd0696068bfea1a42772816021afef097ae77e5a0
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [3]:
spark=SparkSession.builder.appName("Test").getOrCreate()

In [4]:
df1= spark.createDataFrame(data=[(1,"Robert"),(2,"Ria"),(3,"james")],schema='emp_id int,emp_name string')
df2= spark.createDataFrame(data=[(2,"USA"),(4,"iNDIA")],schema='emp_id int,country string')
df1.show()

+------+--------+
|emp_id|emp_name|
+------+--------+
|     1|  Robert|
|     2|     Ria|
|     3|   james|
+------+--------+



In [5]:
df2.show()

+------+-------+
|emp_id|country|
+------+-------+
|     2|    USA|
|     4|  iNDIA|
+------+-------+



In [6]:
df1.join(df2,df1.emp_id == df2.emp_id).show()

+------+--------+------+-------+
|emp_id|emp_name|emp_id|country|
+------+--------+------+-------+
|     2|     Ria|     2|    USA|
+------+--------+------+-------+



In [7]:
df1.join(df2,df1.emp_id == df2.emp_id, "left").show()

+------+--------+------+-------+
|emp_id|emp_name|emp_id|country|
+------+--------+------+-------+
|     1|  Robert|  NULL|   NULL|
|     3|   james|  NULL|   NULL|
|     2|     Ria|     2|    USA|
+------+--------+------+-------+



In [8]:
df1.join(df2,df1.emp_id == df2.emp_id, "left").select(df1.emp_id,df2.country).show()

+------+-------+
|emp_id|country|
+------+-------+
|     1|   NULL|
|     3|   NULL|
|     2|    USA|
+------+-------+



In [9]:
df1.join(df2,df1.emp_id == df2.emp_id, "right").select(df1.emp_id,df2.country).show()

+------+-------+
|emp_id|country|
+------+-------+
|     2|    USA|
|  NULL|  iNDIA|
+------+-------+



In [10]:
df1.join(df2,df1.emp_id == df2.emp_id, "outer").show()

+------+--------+------+-------+
|emp_id|emp_name|emp_id|country|
+------+--------+------+-------+
|     1|  Robert|  NULL|   NULL|
|     2|     Ria|     2|    USA|
|     3|   james|  NULL|   NULL|
|  NULL|    NULL|     4|  iNDIA|
+------+--------+------+-------+



In [14]:
#if we want to get the anti common Data from left table we use {Left_anti, leftanti} join,
#Note: we can't select the columns in right table when using Left_anti

In [13]:
df1.join(df2,df1.emp_id == df2.emp_id, "left_anti").show()

+------+--------+
|emp_id|emp_name|
+------+--------+
|     1|  Robert|
|     3|   james|
+------+--------+



In [16]:
#if we select the column from the right table it throws error

In [15]:
df1.join(df2,df1.emp_id == df2.emp_id, "left_anti").select(df1.emp_id,df2.country).show()

AnalysisException: ignored

In [17]:
#self JOin

In [29]:
df1= spark.createDataFrame(data=[(1,"Robert",2),(2,"Ria",3),(3,"james",5)],schema='emp_id int,emp_name string,managerid int')

df1.show()

+------+--------+---------+
|emp_id|emp_name|managerid|
+------+--------+---------+
|     1|  Robert|        2|
|     2|     Ria|        3|
|     3|   james|        5|
+------+--------+---------+



In [21]:
df1.alias('emp1').join(df1.alias('emp2'),col("emp1.managerid") ==  col("emp2.emp_id"),"inner").show()

+------+--------+---------+------+--------+---------+
|emp_id|emp_name|managerid|emp_id|emp_name|managerid|
+------+--------+---------+------+--------+---------+
|     1|  Robert|        2|     2|     Ria|        3|
|     2|     Ria|        3|     3|   james|        5|
+------+--------+---------+------+--------+---------+



In [30]:
df1.alias('emp1').join(df1.alias('emp2'),col("emp1.managerid") ==  col("emp2.emp_id"),"inner").select(col("emp1.emp_id"),\
                                                                                                      col("emp1.emp_name"),\
                                                                                                      col("emp2.emp_id").alias("managerid"),\
                                                                                                      col("emp2.emp_name")).show()

+------+--------+---------+--------+
|emp_id|emp_name|managerid|emp_name|
+------+--------+---------+--------+
|     1|  Robert|        2|     Ria|
|     2|     Ria|        3|   james|
+------+--------+---------+--------+



In [37]:
#Multi Column and Dataframe Join

In [33]:
df1= spark.createDataFrame(data=[(1,101,"Robert"),(2,102,"Ria"),(3,103,"james")],schema='empid int,deptid int,empname string')
df2= spark.createDataFrame(data=[(2,102,"USA"),(4,104,"INDIA")],schema='empid int,deptid int,country string')
df1.show()

+-----+------+-------+
|empid|deptid|empname|
+-----+------+-------+
|    1|   101| Robert|
|    2|   102|    Ria|
|    3|   103|  james|
+-----+------+-------+



In [34]:
df2.show()

+-----+------+-------+
|empid|deptid|country|
+-----+------+-------+
|    2|   102|    USA|
|    4|   104|  INDIA|
+-----+------+-------+



In [36]:
df1.join(df2,(df1.empid == df2.empid) & (df1.deptid == df2.deptid), 'inner').show()

+-----+------+-------+-----+------+-------+
|empid|deptid|empname|empid|deptid|country|
+-----+------+-------+-----+------+-------+
|    2|   102|    Ria|    2|   102|    USA|
+-----+------+-------+-----+------+-------+



In [43]:
df1= spark.createDataFrame(data=[(1,"Robert"),(2,"Ria"),(3,"james")],schema='empid int,empname string')
df2= spark.createDataFrame(data=[(2,"USA"),(4,"INDIA")],schema='empid int,country string')
df3= spark.createDataFrame(data=[(1,"01-Jan-2023"),(2,"05-Feb-2023"),(3,"07-Mar-2023")],schema='empid int,joindate string')
df1.show()

+-----+-------+
|empid|empname|
+-----+-------+
|    1| Robert|
|    2|    Ria|
|    3|  james|
+-----+-------+



In [39]:
df2.show()

+-----+-------+
|empid|country|
+-----+-------+
|    2|    USA|
|    4|  INDIA|
+-----+-------+



In [40]:
df3.show()

+-----+-----------+
|empid|   joindate|
+-----+-----------+
|    1|01-Jan-2023|
|    2|05-Feb-2023|
|    3|07-Mar-2023|
+-----+-----------+



In [44]:
df1.join(df2,df1.empid == df2.empid,'inner')\
                                            .join(df3,df1.empid == df3.empid,'inner').show()

+-----+-------+-----+-------+-----+-----------+
|empid|empname|empid|country|empid|   joindate|
+-----+-------+-----+-------+-----+-----------+
|    2|    Ria|    2|    USA|    2|05-Feb-2023|
+-----+-------+-----+-------+-----+-----------+

